In [26]:
import os.path
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import pandas as pd
import dlib
import numpy as np

Loading mode, get test_img names etc.

In [27]:
# input labels.
label = pd.read_csv("labels/fairface_label_val.csv")
imgs_path = "fairface-img-margin025-trainval/val"
img_names = [os.path.join(imgs_path, x) for x in os.listdir(imgs_path)]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_fair_7 = torchvision.models.resnet34(pretrained=True)
model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 18)
# if runs on a machine without GPU, should do map_location=torch.device('cpu')
#model_fair_7.load_state_dict(torch.load('fair_face_models/fairface_alldata_20191111.pt')) 
model_fair_7.load_state_dict(torch.load('fair_face_models/fairface_alldata_20191111.pt' , map_location=torch.device('cpu')))
model_fair_7 = model_fair_7.to(device)


# trans

trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/home/chx/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/chx/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Start running the model.

In [28]:
# # Define a hook for all ReLU layers to compute the sum of activations of each filter
# activation = {}
# def relu_hook(module, input, output, layer_name):
#     activation[layer_name] = output.detach()
# # Register the hook for all ReLU layers
# for layer_name, layer in model.named_modules():
#     if isinstance(layer, torch.nn.ReLU):
#         layer.register_forward_hook(relu_hook, layer_name=layer_name)
num_filters = {}
activation = {}
filter_sums = {}
def get_activation(name):
    def hook(model, input, output):
        # Squeeze the tensor to remove the dimension with size 1
        squeezed_tensor = torch.squeeze(output.detach())
        activation[name] = squeezed_tensor
        # If squeezed_tensor has more than one dimension, flatten it
        if len(squeezed_tensor.shape) > 1:
            flattened_tensor = torch.flatten(squeezed_tensor, start_dim=1)
        else:
            flattened_tensor = squeezed_tensor
        # Sum along the second dimension if it exists, otherwise keep the tensor as is
        if flattened_tensor.dim() > 1:
            summed_tensor = flattened_tensor.sum(dim=1)
        else:
            summed_tensor = flattened_tensor
        # Convert the tensor to a list
        summed_list = summed_tensor.tolist()
        filter_sums[name] = summed_list
        num_filters[name] = len(summed_list)
    return hook

for name, layer in model_fair_7.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        layer.register_forward_hook(get_activation(name))
    

In [32]:
# Define class labels for CIFAR-10
classes = ['Black+Female', 'Black+Male', 'East Asian+Female', 'East Asian+Male', 'Indian+Female', 'Indian+Male', 'Latino_Hispanic+Female', 'Latino_Hispanic+Male', 'Middle Eastern+Female', 'Middle Eastern+Male', 'Southeast Asian+Female', 'Southeast Asian+Male', 'White+Female', 'White+Male']
# Define variables to track accuracy and class-wise accuracy
correct = 0
total = 0
class_correct = [0] * 14
class_total = [0] * 14

image_predictation_result = {cl: {} for cl in range(len(classes))}
activation_all_images = []
filtersum_all_images = []
miss_predicated_images = []
# Set the model to evaluation mode
model_fair_7.eval()
my_label = pd.read_csv("labels/fairface_label_val.csv")
for index, img_name in enumerate(img_names):
    if index % 1000 == 0:
        print("Predicting... {}/{}".format(index, len(img_names)))
        img_name_in_label =  'val/' +img_name.split('/')[-1]
        label_row = my_label[my_label['file'] == img_name_in_label].iloc[0]
        
        # load image, run model.
        image = dlib.load_rgb_image(img_name)
        image = trans(image)
        image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
        image = image.to(device)
        
        outputs = model_fair_7(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)
        
        race_outputs = outputs[:7]
        gender_outputs = outputs[7:9]
        age_outputs = outputs[9:18]
        
        race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
        gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
        age_score = np.exp(age_outputs) / np.sum(np.exp(age_outputs))

        race_pred = np.argmax(race_score)
        gender_pred = np.argmax(gender_score)
        age_pred = np.argmax(age_score)

        race_dict = {
            0: 'White',
            1: 'Black',
            2: 'Latino_Hispanic',
            3: 'East Asian',
            4: 'Southeast Asian',
            5: 'Indian',
            6: 'Middle Eastern'
        }
        gender_dict = { 0:'Male' , 1 :'Female'}
        
        print(race_pred,gender_pred,age_pred)

Predicting... 0/10954
5 0 3
Predicting... 1000/10954
4 0 3
Predicting... 2000/10954
3 0 7
Predicting... 3000/10954
0 1 3
Predicting... 4000/10954
3 1 3
Predicting... 5000/10954
5 0 3
Predicting... 6000/10954
1 1 4
Predicting... 7000/10954
1 1 4
Predicting... 8000/10954
4 1 3
Predicting... 9000/10954
2 1 3
Predicting... 10000/10954
1 1 1
